In [47]:
import base64
import random
from utils.get_save_base64 import decode_image, encode_image
from utils.get_postgres_connection import _conn_open

from rich.console import Console

console = Console()
conn = _conn_open()

Conn: <connection object at 0x0000026478ECD140; dsn: 'user=postgres password=xxx dbname=postgres 
host=host.docker.internal', closed: 0>

In [48]:
encoded_image_data = encode_image("test02.png").decode("utf-8")

In [49]:
print(encoded_image_data)

...


In [50]:
sql = f"""
INSERT INTO tbl_doc_elements (
	file_id, element_id, element_text, image_base64, embedding)
VALUES 
('43524c61-ef85-404e-b487-fb0fe304d8e2','1043','43 Hello World',
'{encoded_image_data}', ARRAY[1, 2])
RETURNING id;
"""

In [51]:
# console.print(sql)

In [52]:
cur = conn.cursor()
try:
    cur.execute(sql)
    row = cur.fetchone()
    id = row[0]
    console.print(f"ID is {id}")
    conn.commit()
except Exception as e:
    console.print(e)

ID is e726a5a8-5245-402b-9e55-4afb0780794d

In [53]:
cur = conn.cursor()

sql = f"SELECT id, image_base64 FROM tbl_doc_elements WHERE id='{id}';"
cur.execute(sql)
rows = cur.fetchall()
if not len(rows):
    console.print("empty")
for row in rows:
    
    encoded_image_data = row[1]
    # console.print(encoded_image_data)
cur.close()
conn.close()

In [54]:
output_filepath = f"test_output_{random.randint(100, 900)}.png"
print(output_filepath)
x = decode_image(encoded_image_data, output_filepath)

test_output_243.png
